In [3]:
!pip install gmaps

In [4]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [26]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
# Store csv created in part one into a DataFrame
weather_report = pd.read_csv("output_data/cities.csv")
weather_report.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [34]:
# Heatmap of humidity
#Create a heat map that displays the humidity for every city from the part I of the homework.

#Locate city with lag and lng
city = weather_report[["Lat", "Lng"]]
humidity = weather_report["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(city, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)


#save fig as png
plt.savefig("output_data/heatmap.png")

plt.show()

# Display figure
fig


<Figure size 432x288 with 0 Axes>

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
#Max Temp < 80 and > 70
#Wind Speed < 10MPH
#Cloudiness = 0

selective_df = weather_report.loc[(weather_report["Wind Speed"] < 10) & (weather_report["Cloudiness"] == 0) & (weather_report["Max Temp"] > 70) & (weather_report["Max Temp"] < 80)].dropna()

selective_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates

hotel_df = selective_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add empty column to data frame
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [11]:
# Set parameters to search for a hotel

params = {
    "type": "hotel",
    "keyword": "hotel",
    "radius": 3000,
    "key": g_key,
}

# Iterate through
#Iterrows coding help from: https://www.reddit.com/r/learnpython/comments/7qzm2s/attempting_to_loop_over_a_df_and_plot_but_having/
for index, row in hotel_df.iterrows(): 
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel"
        }
        
        
    # make request and print url
    response = requests.get(base_url, params)
    hotels = response.json()
    
    # convert to json
    print(json.dumps(hotels, indent=4, sort_keys=True))
    
    # Grab the first hotel from the results and store the name
    print(hotels["results"][0]["name"])
    
    #add try except to account for missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

{
    "error_message": "You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}


IndexError: list index out of range

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map

# Create a marker_layer
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


#save fig as png
plt.savefig('output_data/gmap.png')

plt.show()

# Display figure
fig


<Figure size 432x288 with 0 Axes>

Figure(layout=FigureLayout(height='420px'))